In [1]:
import src.domain as domain
import src.repository as repository


In [2]:
import src.repository as repository

driver = repository.get_driver_fanduel_live_odds_page_existing_chrome()

Error clicking Live now: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff74efdfe75+79173]
	GetHandleVerifier [0x0x7ff74efdfed0+79264]
	(No symbol) [0x0x7ff74ed99e5a]
	(No symbol) [0x0x7ff74edf0586]
	(No symbol) [0x0x7ff74edf083c]
	(No symbol) [0x0x7ff74ee44247]
	(No symbol) [0x0x7ff74ee189af]
	(No symbol) [0x0x7ff74ee4100d]
	(No symbol) [0x0x7ff74ee18743]
	(No symbol) [0x0x7ff74ede14c1]
	(No symbol) [0x0x7ff74ede2253]
	GetHandleVerifier [0x0x7ff74f2aa2ad+3004797]
	GetHandleVerifier [0x0x7ff74f2a46fd+2981325]
	GetHandleVerifier [0x0x7ff74f2c3350+3107360]
	GetHandleVerifier [0x0x7ff74effa9fe+188622]
	GetHandleVerifier [0x0x7ff74f00228f+219487]
	GetHandleVerifier [0x0x7ff74efe8dc4+115860]
	GetHandleVerifier [0x0x7ff74efe8f79+116297]
	GetHandleVerifier [0x0x7ff74efcf528+11256]
	BaseThreadInitThunk [0x0x7ffd2cfae8d7+23]
	RtlUserThreadStart [0x0x7ffd2e1fc5dc+44]

Error clicking Baseball: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff74efdfe75+79173]
	GetHandleVerifier [0x0x7ff74efdfed0+7

In [3]:
import undetected_chromedriver as uc
import time


# Create undetected Chrome driver instance
driver = uc.Chrome()

# Navigate to the target URL
driver.get("https://sportsbook.fanduel.com/")

KeyboardInterrupt: 

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    wait = WebDriverWait(driver, 10)
    live_now = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Live now']")))
    live_now.click()
except Exception as e:
    print(f"Error clicking Live now: {e}")


In [ ]:
try:
    wait = WebDriverWait(driver, 10)
    baseball = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Baseball']")))
    baseball.click()
except Exception as e:
    print(f"Error clicking Baseball: {e}")


In [6]:
res = src.repository.parse_live_odds()

In [7]:
res

{'Hiroshima Toyo Carp': '-750', 'Chiba Lotte Marines': '+460'}

In [4]:
import src.domain as domain

odds = domain.Odds("-2000")
print(odds)



-2000 - 0.9523809523809523
